<a href="https://colab.research.google.com/github/wildmary/hello/blob/master/%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re # ну регулярки нам всегда нужны
import requests
import csv
from lxml import etree
import pandas as pd # пандас традиционно импортируют с алиасом pd, будем придерживаться
pd.__version__ # посмотрим версию -- вроде в колабе довольно свежая

'1.1.5'

In [ ]:
part = pd.read_csv("part.csv", sep=";")
#partxlsx  = pd.read_excel("part.xlsx")
part.columns = ["Griechisch", "GLemma", "GMSA", "Slavisch", "SÜbersetzung", "SemantK", "Lateinisch", "LLemma", "LGram"]

In [185]:
part[:13]
#part["SemantK"][12] = "тожде єже Βαθμίς"


,Griechisch,GLemma,GMSA,Slavisch,SÜbersetzung,SemantK,Lateinisch,LLemma,LGram
0,Hyper-/Metalemma,Lemma,МSА,Hyper-/Metalemma,Übersetzung,Semantischer Kommentar,Hyper-/Metalemma,Lemma,Gramm. Inf.
1,NaN,Βαθμηδόν,NaN,NaN,"стꙋпнѡ,",NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,\степеⷩѡ/,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,степенⷩическы,NaN,gradatim,gradatim·,NaN
4,NaN,Βαθμίς,"ιδος, ἡ",NaN,степень,NaN,gradus,gradus·,NaN
5,NaN,NaN,NaN,NaN,стопа,NaN,vestigium,vestigium pedis·,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,pes,NaN,NaN
7,NaN,NaN,NaN,NaN,сѣдалище,NaN,NaN,sedes·,NaN
8,NaN,NaN,NaN,NaN,ѡснованіе,NaN,fundamentum,fundamentũ·,NaN
9,NaN,NaN,NaN,NaN,прагᲆ,NaN,limen,limen·,NaN


In [ ]:
#tei = (open('tei.xml','w'))
#tei.write(etree.tostring(root, encoding="unicode"))
with open ('tei.xml', 'w', encoding = 'utf-8') as tei:  
  tei.write (etree.tostring(root, encoding='unicode'))

In [ ]:
root = etree.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0") # создадим корневой тег


In [186]:
teiheader = etree.Element("teiHeader")
root.append(teiheader)
fileDesc = etree.SubElement(teiheader, "fileDesc")
titleStmt = etree.SubElement(fileDesc, "titleStmt")
title = etree.SubElement(titleStmt, "title")
title.text = 'Словарь Епифания Славинецкого'

In [ ]:
table = []
for i, row in part[:12].iterrows():
  print("текущий", list(row))
  previous = list(row)
  table.append(previous)
  #print(table[0])
  if i == 0:
    continue
  else:
    table.remove(table[0])
  print("предыдущий", table[0][1])
 
  

In [187]:
root = etree.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0") # создадим корневой тег
teiheader = etree.Element("teiHeader")
root.append(teiheader)
fileDesc = etree.SubElement(teiheader, "fileDesc")
titleStmt = etree.SubElement(fileDesc, "titleStmt")
title = etree.SubElement(titleStmt, "title")
title.text = 'Словарь Епифания Славинецкого'
text = etree.SubElement(root, "text")
body = etree.SubElement(text, "body")
for index, row in part[:13].iterrows():
  if index == 0:
    continue
  else:
   # print("следующий", part['SÜbersetzung'][index+1])
    if type(row["Griechisch"]) != float: #надо объединить с леммой на случай если есть гиперлемма
      #print(row["Griechisch"])
      entry = etree.SubElement(body, "entry", xml_id=row["GLemma"], xml_lang="grc")
      form = etree.SubElement(entry, "form", type="hyperlemma")
      orth = etree.SubElement(form, "orth")
      orth.text = row["Griechisch"]
    #else: 
    if type(row["GLemma"]) != float:
      print(row["GLemma"])
      if type(row["Griechisch"]) == float:
        entry = etree.SubElement(body, "entry", xml_id=row["GLemma"], xml_lang="grc")
      form = etree.SubElement(entry, "form", type="lemma")
      orth = etree.SubElement(form, "orth")
      orth.text = row["GLemma"] #убрать запятые, иногда встречаются!
      #print(type(row["GMSA"]))
      if type(row["GMSA"]) != float: #доделать!
        inflected = re.search("(.*?),\s(.*)", row["GMSA"])
        if inflected:
          inflected_gram()
      if type(row["SÜbersetzung"]) != float:
         sense_greek = etree.SubElement(entry, "sense", xml_id=row["GLemma"]+".1")
   #     print("текущий",row["GLemma"], row["SÜbersetzung"])
         slavic_latin_translation()
      several_slavic_equivalents_check()
    if type(row["SemantK"]) != float:
      sense_note = etree.SubElement(entry, "sense", xml_id=row["GLemma"]+".1")
      note = etree.SubElement(sense_note, "note")
      print(row["SemantK"])
      if row["SemantK"] == "тожде":
        xr = etree.SubElement(note, "xr", type="synonymy")
        while type(part["GLemma"][index-1]) == float:
          index-= 1
        ref = etree.SubElement(xr, "ref", type="entry", target="#"+part['GLemma'][index-1])
        ref.text = row["SemantK"]
      cross_reference_mention = re.search("(тожде єже|тожде)\s(.*)", row["SemantK"])
      if cross_reference_mention:
        xr = etree.SubElement(note, "xr", type="synonymy")
        lbl = etree.SubElement(xr, "lbl")
        lbl.text = cross_reference_mention.group(1)
        ref = etree.SubElement(xr, "ref", type="entry", target="#"+cross_reference_mention.group(2))
        ref.text = cross_reference_mention.group(2)


        



Βαθμηδόν
Βαθμίς
Βαθμός
тожде єже Βαθμίς


In [188]:
print(etree.tostring(root, pretty_print=True, encoding='unicode'))

<TEI xmlns="http://www.tei-c.org/ns/1.0">
  <teiHeader>
    <fileDesc>
      <titleStmt>
        <title>Словарь Епифания Славинецкого</title>
      </titleStmt>
    </fileDesc>
  </teiHeader>
  <text>
    <body>
      <entry xml_id="Βαθμηδόν" xml_lang="grc">
        <form type="lemma">
          <orth>Βαθμηδόν</orth>
        </form>
        <sense xml_id="Βαθμηδόν.1">
          <cit type="translationEquivalent" xml_lang="chu">
            <form xml_id="стꙋпнѡ,">
              <orth>стꙋпнѡ,</orth>
            </form>
          </cit>
          <cit type="translationEquivalent" xml_lang="chu">
            <form xml_id="\степеⷩѡ/">
              <orth>\степеⷩѡ/</orth>
            </form>
          </cit>
          <cit type="translationEquivalent" xml_lang="chu">
            <form xml_id="степенⷩическы">
              <orth>степенⷩическы</orth>
            </form>
            <sense xml_id="степенⷩическы.1">
              <cit type="translationEquivalent" xml_lang="lat">
                <

In [ ]:
text = etree.SubElement(root, "text")


In [ ]:
 def slavic_latin_translation():
   #sense_greek = etree.SubElement(entry, "sense", xml_id=row["GLemma"]+".1")
   cit = etree.SubElement(sense_greek, "cit", type="translationEquivalent", xml_lang = "chu")
   form = etree.SubElement(cit, "form",  xml_id=row["SÜbersetzung"])
   orth = etree.SubElement(form, "orth")
   orth.text = row["SÜbersetzung"]
   if type(row["Lateinisch"]) != float:
        sense = etree.SubElement(cit, "sense", xml_id=row["SÜbersetzung"]+".1")
        cit = etree.SubElement(sense, "cit", type="translationEquivalent", xml_lang = "lat")
        form = etree.SubElement(cit, "form", type="hyperlemma", xml_id=row["Lateinisch"])
        orth = etree.SubElement(form, "orth")
        orth.text = row["Lateinisch"]
   if type(row["LLemma"]) != float:
    form = etree.SubElement(cit, "form", type="lemma", xml_id=row["LLemma"])
    orth = etree.SubElement(form, "orth")
    orth.text = row["LLemma"]

   
    

In [ ]:
def inflected_gram():
  form = etree.SubElement(entry, "form", type="inflected")
  gramGrp = etree.SubElement(form, "gramGrp")
  gram = etree.SubElement(gramGrp, "gram", case="genitiv")
  orth = etree.SubElement(form, "orth", extent="part")
  orth.text = inflected.group(1)
  if inflected.group(2) == "ἡ":
     gramGrp = etree.SubElement(entry, "gramGrp")
     gram = etree.SubElement(gramGrp, "gram", gen="fem")
     gram.text = "ἡ"
  elif inflected.group(2) == "τὸ":
     gramGrp = etree.SubElement(entry, "gramGrp")
     gram = etree.SubElement(gramGrp, "gram", gen="neut")
     gram.text = "τὸ"
  else:
     gramGrp = etree.SubElement(entry, "gramGrp")
     gram = etree.SubElement(gramGrp, "gram", gen="masc")
     gram.text = "ὁ"

In [ ]:
def several_slavic_equivalents_check():
  global index
  while type(part["GLemma"][index+1]) == float:
    if type(part['SÜbersetzung'][index+1]) != float:
      #print("есть еще перевод", "--", part['SÜbersetzung'][index+1])
      cit = etree.SubElement(sense_greek, "cit", type="translationEquivalent", xml_lang = "chu")
      form = etree.SubElement(cit, "form",  xml_id=part['SÜbersetzung'][index+1])
      orth = etree.SubElement(form, "orth")
      orth.text = part['SÜbersetzung'][index+1]
      if type(part['Lateinisch'][index+1]) != float:
       # print("есть гиперлемма к", part['SÜbersetzung'][index+1], part['Lateinisch'][index+1])
        sense = etree.SubElement(cit, "sense", xml_id=part['SÜbersetzung'][index+1]+".1")
        cit = etree.SubElement(sense, "cit", type="translationEquivalent", xml_lang = "lat")
        form = etree.SubElement(cit, "form", type="hyperlemma", xml_id=part['Lateinisch'][index+1])
        orth = etree.SubElement(form, "orth")
        orth.text = part['Lateinisch'][index+1]
      if typepart['LLemma'][index+1]() != float:
        if type(part['Lateinisch'][index+1]) == float:
          sense = etree.SubElement(cit, "sense", xml_id=part['SÜbersetzung'][index+1]+".1")
          cit = etree.SubElement(sense, "cit", type="translationEquivalent", xml_lang = "lat")
        #print("есть лемма", part['LLemma'][index+1])
        form = etree.SubElement(cit, "form", type="lemma", xml_id=part['LLemma'][index+1])
        orth = etree.SubElement(form, "orth")
        orth.text = part['LLemma'][index+1]
    index+= 1